In [3]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

#### 最初

In [13]:
data_in_HERB = pd.read_csv('Herb_Compound_in_HERB 0202.csv',index_col=0)
comp_info_in_HERB = pd.read_csv(r"D:\Herb数据库\信息表\HERB_ingredient_info.txt",sep='\t')
herb_info_in_HERB = pd.read_csv(r"D:\Herb数据库\信息表\HERB_herb_info.txt",sep='\t')
herb_info_in_HERB.rename(columns={'Herb_ID':'Herb_id'},inplace=True)
data_internal = pd.read_excel('Herb_Compound_internal 0130.xlsx',index_col=0)
positive_pair_with_herb = pd.read_csv(r"D:\Herb数据库\阳性关联(文献报道的药材或成分-疾病有关联的数据).csv",index_col=0)

In [49]:
positive_herb = list(positive_pair_with_herb['Herb_or_Ingredient_id'])
data_need_to_add = pd.merge(data_in_HERB.query("Herb_id in @positive_herb"),herb_info_in_HERB[['Herb_id','Herb_cn_name']],on='Herb_id',how='left')
data_need_to_add = pd.merge(data_need_to_add,comp_info_in_HERB[['Ingredient_id','Ingredient_Smile']],
                            on='Ingredient_id',how='left').query("Ingredient_Smile != 'Not Available'").dropna()

In [52]:
herb_less_than_10_ingre_in_HERB = list(pd.DataFrame(data_need_to_add.groupby('Herb_id')['Ingredient_id'].count()).query("Ingredient_id<10").index)
data_need_to_add = data_need_to_add.query("Herb_id not in @herb_less_than_10_ingre_in_HERB")

In [53]:
data_need_to_add.to_excel("Herb_Compound_need_to_add_in_HERB.xlsx")

In [54]:
set(data_need_to_add['Herb_cn_name'])

{'丹参',
 '姜黄',
 '杏仁',
 '橄榄',
 '灯盏细辛',
 '灵芝',
 '生姜',
 '白术',
 '积雪草',
 '紫草',
 '肉桂',
 '菊花',
 '葛根',
 '蘑菇',
 '螺旋藻',
 '辣椒',
 '金银花',
 '银杏叶',
 '雷公藤',
 '鱼腥草',
 '黄芩',
 '黄芪',
 '黄连',
 '龙胆',
 '龙葵'}

In [21]:
data_internal[data_internal['Herb'].str.contains('忍冬')]

,Herb,smiles,分离部位,Canonical_smiles
21634,忍冬,O=C(O)c1ccc(O)c(O)c1,花蕾,O=C(O)C1=CC=C(O)C(O)=C1
21635,忍冬,COc1cc(/C=C/C(=O)O)ccc1O,花蕾,COC1=CC(C=CC(=O)O)=CC=C1O
21636,忍冬,CCO/C=C/C(=O)O,花蕾,CCOC=CC(=O)O
21637,忍冬,C[C@@H]1O[C@@H](O[C@H]2[C@H](Oc3cc(O)c4c(=O)cc...,叶,CC1OC(OC2C(OC3=CC(O)=C4C(=O)C=C(C5=CC=C(O)C(O)...
21638,忍冬,O=C(/C=C/c1ccc(O)c(O)c1)O[C@@H]1C[C@](O)(C(=O)...,叶/根,O=C(C=CC1=CC=C(O)C(O)=C1)OC1CC(O)(C(=O)O)CC(OC...
...,...,...,...,...
21831,忍冬,O=c1cc(-c2ccc(Oc3cc(-c4cc(=O)c5c(O)cc(O)cc5o4)...,叶,O=C1C=C(C2=CC=C(OC3=CC(C4=CC(=O)C5=C(O)C=C(O)C...
21832,忍冬,O=c1cc(-c2ccc(Oc3cc(-c4cc(=O)c5c(O)cc(O[C@@H]6...,叶,O=C1C=C(C2=CC=C(OC3=CC(C4=CC(=O)C5=C(O)C=C(OC6...
21833,忍冬,O=c1cc(-c2ccc(O)cc2)oc2cc(O[C@@H]3O[C@H](CO)[C...,叶,O=C1C=C(C2=CC=C(O)C=C2)OC2=CC(OC3OC(CO)C(O)C(O...
21834,忍冬,COc1ccc(-c2cc(=O)c3c(O)cc(O[C@@H]4O[C@H](CO)[C...,叶,COC1=CC=C(C2=CC(=O)C3=C(O)C=C(OC4OC(CO)C(O)C(O...


#### 经过数据清洗后的数据 0207

In [72]:
data_internal_new = pd.read_excel('Herb_Compound_internal 0206.xlsx')

In [57]:
data_need_to_add.rename(columns={'Herb_cn_name':'Herb','Ingredient_Smile':'smiles'},inplace=True)

In [68]:
can_smi = []
for smi in data_need_to_add['smiles']:
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
    except:
        can_smi.append(None)
        print(smi)
data_need_to_add['Canonical_smiles'] = can_smi

None
C29H45NO5
CC1=CC=C(C=C1)C(CO
None
None


In [78]:
data_all = pd.concat([data_internal_new[['Herb','Canonical_smiles']],data_need_to_add[['Herb','Canonical_smiles']]],axis=0,ignore_index=True).dropna().drop_duplicates()

In [93]:
less_10_herb = list(pd.DataFrame(data_all.groupby('Herb').count()).query('Canonical_smiles<10').index)
data_all1 = data_all.query("Herb not in @less_10_herb")
data_all1.to_csv("Herb_Compound_all 0207.csv")

In [95]:
len(set(data_all1['Herb'])),len(set(data_all1['Canonical_smiles']))

(287, 13435)

#### 加上银杏补充的成分

In [3]:
data_internal_new = pd.read_excel(r'Herb_Compound_internal 0215.xlsx',index_col=0)
data_need_to_add = pd.read_excel(r"Herb_Compound_need_to_add_in_HERB.xlsx",index_col=0)
data_need_to_add.rename(columns={'Herb_cn_name':'Herb','Ingredient_Smile':'smiles'},inplace=True)

In [4]:
can_smi = []
for smi in data_need_to_add['smiles']:
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
    except:
        can_smi.append(None)
        print(smi)
data_need_to_add['Canonical_smiles'] = can_smi

None
C29H45NO5
CC1=CC=C(C=C1)C(CO
None
None


In [5]:
can_smi = []
for smi in data_internal_new['smiles']:
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
    except:
        can_smi.append(None)
        print(smi)
data_internal_new['Canonical_smiles'] = can_smi

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [9]:
data_all = pd.concat([data_internal_new[['Herb','Canonical_smiles']],data_need_to_add[['Herb','Canonical_smiles']]],axis=0,ignore_index=True).dropna().drop_duplicates()
data_all.to_csv("Herb_Compound_all 0215.csv")

In [8]:
len(set(data_all['Herb'])),len(set(data_all['Canonical_smiles']))

(295, 13499)

In [5]:
all_smiles = pd.DataFrame(list(set(pd.read_csv("Herb_Compound_all 0215.csv")['Canonical_smiles']))).to_csv("all_smiles 0228.csv")

In [4]:
len(all_smiles)

13499

#### 加上inchikey

In [5]:
data_all = pd.read_csv("Herb_Compound_all 0215.csv",index_col=0)
inchikey = []
for smi in tqdm(data_all['Canonical_smiles']):
    try:
        inchikey.append(Chem.MolToInchiKey(Chem.MolFromSmiles(smi)))
    except:
        inchikey.append(None)
        print(smi)
data_all['inchikey'] = inchikey

100%|██████████| 22190/22190 [00:09<00:00, 2265.76it/s]


In [9]:
data_all.loc[data_all[['Herb','inchikey']].drop_duplicates().index].to_csv("Herb_Compound_all 0316.csv")

In [4]:
data_all = pd.read_csv("Herb_Compound_all 0316.csv",index_col=0)

In [6]:
len(set(data_all['inchikey'])),len(set(data_all['Herb']))

(13183, 295)